### An Application that uses LLM ability to discern important information instead of saving everything in the conversation history

Demonstrates tool calling, advanced use of context restoration

In [71]:
from openai import OpenAI
import os
import gradio as gr
import sqlite3
import json

In [72]:
openai= OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openrouter.ai/api/v1",  
)

In [73]:
gpt_model = "gpt-4.1-mini"
claude_model = "anthropic/claude-3.5-haiku"

Specifiy in the system prompt that LLM should store and read from the db

In [74]:
system_prompt = """
You are a helpful assistant that explains code snippets.

At the start of every conversation you MUST call a tool to load past messages.
After generating a response, you MUST call a tool to store:
- assistant_message
- user_message
"""

In [75]:
DB = "convo.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS conversations (id INTEGER PRIMARY KEY AUTOINCREMENT, created_at DATETIME DEFAULT CURRENT_TIMESTAMP, user_message TEXT, assistant_message TEXT)')
    conn.commit()

In [76]:
def write_history_todb(assistant_message, user_message):
    print("DATABASE TOOL CALLED: Writing history to db", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO conversations (user_message, assistant_message) VALUES (?, ?)",
            (user_message, assistant_message),
        )
        conn.commit()


def read_history_from_db():
    print("DATABASE TOOL CALLED: Reading history from db", flush=True)
    messages = []

    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT user_message, assistant_message FROM conversations")
        rows = cursor.fetchall()

        if not rows:
            return []  # empty DB case

        for user_message, assistant_message in rows:
            messages.append({"role": "user", "content": user_message})
            messages.append({"role": "assistant", "content": assistant_message})

        return messages

In [77]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "write_history_todb":
            arguments = json.loads(tool_call.function.arguments)
            write_history_todb(
                arguments["assistant_message"], arguments["user_message"]
            )
            responses.append(
                {
                    "role": "tool",
                    "content": "History written to db",
                    "tool_call_id": tool_call.id,
                }
            )
        elif tool_call.function.name == "read_history_from_db":
            history_messages = read_history_from_db()
            responses.append(
                {
                    "role": "tool",
                    "content": json.dumps(history_messages),
                    "tool_call_id": tool_call.id,
                }
            )
    return responses

In [78]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "write_history_todb",
            "description": "Write the conversation history to the database",
            "parameters": {
                "type": "object",
                "properties": {
                    "assistant_message": {"type": "string"},
                    "user_message": {"type": "string"},
                },
                "required": ["assistant_message", "user_message"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "read_history_from_db",
            "description": "Read the conversation history from the database",
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
    }
]

In [79]:
def chat(message, history):
    # load history from db
    history = read_history_from_db()
    messages = (
        [{"role": "system", "content": system_prompt}]
        + history
        + [{"role": "user", "content": message}]
    )
    response = openai.chat.completions.create(
        model=gpt_model, messages=messages, tools=tools
    )

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(
            model=gpt_model, messages=messages, tools=tools
        )

    return response.choices[0].message.content


In [80]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Writing history to db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Reading history from db
DATABASE TOOL CALLED: Writing history to db
DATABASE TOOL CALLED: Writing history to db
